# Data Preparation 

In [1]:
# Importing libraries

import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load dataset
df = pd.read_csv("data/preprocessed/main.csv")
df.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Dept,Weekly_Sales,Store_Type,Store_Size
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,1,24924.50,A,151315
1,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,2,50605.27,A,151315
2,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,3,13740.12,A,151315
3,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,4,39954.04,A,151315
4,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,5,32229.38,A,151315


In [3]:
# # Remove all MarkDown columns (Excluded from the analysis)
markdown_columns = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
df = df.drop(columns=markdown_columns)
print(df.head())

   Store        Date  Temperature  Fuel_Price         CPI  Unemployment  \
0      1  2010-02-05        42.31       2.572  211.096358         8.106   
1      1  2010-02-05        42.31       2.572  211.096358         8.106   
2      1  2010-02-05        42.31       2.572  211.096358         8.106   
3      1  2010-02-05        42.31       2.572  211.096358         8.106   
4      1  2010-02-05        42.31       2.572  211.096358         8.106   

   IsHoliday  Dept  Weekly_Sales Store_Type  Store_Size  
0      False     1      24924.50          A      151315  
1      False     2      50605.27          A      151315  
2      False     3      13740.12          A      151315  
3      False     4      39954.04          A      151315  
4      False     5      32229.38          A      151315  


# Feature Engineering

### Transform Temporal Data
- Create Date-Based Features

In [4]:
df['Month'] = pd.to_datetime(df['Date']).dt.month
df['Day'] = pd.to_datetime(df['Date']).dt.day
df['Week'] = pd.to_datetime(df['Date']).dt.isocalendar().week
df = df.drop(columns=['Date'])  # Drop the original date column
df.head()

,Store,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday,Dept,Weekly_Sales,Store_Type,Store_Size,Month,Day,Week
0,1,42.31,2.572,211.096358,8.106,False,1,24924.50,A,151315,2,5,5
1,1,42.31,2.572,211.096358,8.106,False,2,50605.27,A,151315,2,5,5
2,1,42.31,2.572,211.096358,8.106,False,3,13740.12,A,151315,2,5,5
3,1,42.31,2.572,211.096358,8.106,False,4,39954.04,A,151315,2,5,5
4,1,42.31,2.572,211.096358,8.106,False,5,32229.38,A,151315,2,5,5


### Encode Categorical Variables
Encoding categorical variables using One-Hot Encoding

In [5]:
df = pd.get_dummies(df, columns=['Store_Type', 'Store'])
df.head()

,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday,Dept,Weekly_Sales,Store_Size,Month,Day,...,Store_36,Store_37,Store_38,Store_39,Store_40,Store_41,Store_42,Store_43,Store_44,Store_45
0,42.31,2.572,211.096358,8.106,False,1,24924.50,151315,2,5,...,False,False,False,False,False,False,False,False,False,False
1,42.31,2.572,211.096358,8.106,False,2,50605.27,151315,2,5,...,False,False,False,False,False,False,False,False,False,False
2,42.31,2.572,211.096358,8.106,False,3,13740.12,151315,2,5,...,False,False,False,False,False,False,False,False,False,False
3,42.31,2.572,211.096358,8.106,False,4,39954.04,151315,2,5,...,False,False,False,False,False,False,False,False,False,False
4,42.31,2.572,211.096358,8.106,False,5,32229.38,151315,2,5,...,False,False,False,False,False,False,False,False,False,False


### Standardize/Scale Features
Standardize numerical features like Temperature, Fuel_Price, CPI, and Unemployment to improve model performance

In [6]:
scaler = StandardScaler()

# Define columns to scale
columns_to_scale = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Store_Size']

df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

df.head()

,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday,Dept,Weekly_Sales,Store_Size,Month,Day,...,Store_36,Store_37,Store_38,Store_39,Store_40,Store_41,Store_42,Store_43,Store_44,Store_45
0,-0.963831,-1.72052,1.018422,0.078331,False,1,24924.50,0.238802,2,5,...,False,False,False,False,False,False,False,False,False,False
1,-0.963831,-1.72052,1.018422,0.078331,False,2,50605.27,0.238802,2,5,...,False,False,False,False,False,False,False,False,False,False
2,-0.963831,-1.72052,1.018422,0.078331,False,3,13740.12,0.238802,2,5,...,False,False,False,False,False,False,False,False,False,False
3,-0.963831,-1.72052,1.018422,0.078331,False,4,39954.04,0.238802,2,5,...,False,False,False,False,False,False,False,False,False,False
4,-0.963831,-1.72052,1.018422,0.078331,False,5,32229.38,0.238802,2,5,...,False,False,False,False,False,False,False,False,False,False


In [7]:
# Save the ML ready dataset
df.to_csv('./data/preprocessed/main_ML_ready.csv', index=False, header=True)